# Exemplo Completo - Rede Neural para Classificação de Imagens

Este notebook demonstra passo a passo como usar o projeto de redes neurais.

## Conteúdo:
1. Configuração do ambiente
2. Carregamento de dados
3. Exploração e visualização
4. Preprocessamento
5. Criação do modelo
6. Treinamento
7. Avaliação
8. Salvamento e carregamento do modelo

## 1. Configuração do Ambiente

In [ ]:
# Importações necessárias
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

# Adiciona o diretório src ao path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Importa nossos módulos
from data_loader import (
    carregar_mnist, carregar_fashion_mnist, carregar_cifar10,
    preprocessar_dados, dividir_validacao
)
from model import (
    criar_modelo_simples, criar_modelo_cnn,
    compilar_modelo, resumo_modelo
)
from train import (
    criar_callbacks, treinar_modelo,
    plotar_historico
)
from evaluate import (
    avaliar_modelo, fazer_predicoes,
    matriz_confusao, visualizar_predicoes
)

# Configurações
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

# Verifica GPU
print(f"TensorFlow versão: {tf.__version__}")
print(f"GPUs disponíveis: {len(tf.config.list_physical_devices('GPU'))}")

## 2. Carregamento de Dados

Vamos carregar o dataset MNIST como exemplo. Você pode trocar facilmente por outros datasets.

In [ ]:
# Carrega o dataset (escolha um)
# (x_train, y_train), (x_test, y_test) = carregar_mnist()
(x_train, y_train), (x_test, y_test) = carregar_fashion_mnist()
# (x_train, y_train), (x_test, y_test) = carregar_cifar10()

# Informações sobre o dataset
print(f"\nFormato dos dados:")
print(f"  x_train: {x_train.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  x_test: {x_test.shape}")
print(f"  y_test: {y_test.shape}")
print(f"\nTipo de dados: {x_train.dtype}")
print(f"Valores mín/máx: {x_train.min()}/{x_train.max()}")

## 3. Exploração e Visualização

In [ ]:
# Visualiza algumas amostras
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.ravel()

# Para Fashion-MNIST
class_names = ['Camiseta', 'Calça', 'Suéter', 'Vestido', 'Casaco',
               'Sandália', 'Camisa', 'Tênis', 'Bolsa', 'Bota']

for i in range(10):
    idx = np.random.randint(0, len(x_train))
    img = x_train[idx]
    label = y_train[idx]
    
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(f'{class_names[label]} ({label})')
    axes[i].axis('off')

plt.suptitle('Amostras do Dataset', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Distribuição das classes
unique, counts = np.unique(y_train, return_counts=True)

plt.figure(figsize=(10, 6))
plt.bar(unique, counts)
plt.xlabel('Classe')
plt.ylabel('Quantidade')
plt.title('Distribuição das Classes no Conjunto de Treino')
plt.xticks(unique, [class_names[i] for i in unique], rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Classes balanceadas: {np.allclose(counts, counts[0])}")

## 4. Preprocessamento

In [ ]:
# Preprocessa os dados
x_train_prep, x_test_prep = preprocessar_dados(
    x_train, x_test,
    normalizar=True,
    expandir_dims=True  # Para CNN
)

# Divide em treino/validação
x_train_final, y_train_final, x_val, y_val = dividir_validacao(
    x_train_prep, y_train,
    val_split=0.1
)

print(f"\nDados após preprocessamento:")
print(f"  Treino: {x_train_final.shape}")
print(f"  Validação: {x_val.shape}")
print(f"  Teste: {x_test_prep.shape}")
print(f"\nValores mín/máx após normalização: {x_train_final.min():.2f}/{x_train_final.max():.2f}")

## 5. Criação do Modelo

In [ ]:
# Define parâmetros do modelo
input_shape = x_train_final.shape[1:]  # (28, 28, 1) para MNIST/Fashion
num_classes = 10

# Cria o modelo (escolha um)
# modelo = criar_modelo_simples(input_shape, num_classes)
modelo = criar_modelo_cnn(input_shape, num_classes)

# Compila o modelo
modelo = compilar_modelo(
    modelo,
    learning_rate=0.001,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Mostra resumo do modelo
resumo_modelo(modelo)

In [ ]:
# Visualiza arquitetura do modelo
tf.keras.utils.plot_model(
    modelo,
    to_file='../results/model_architecture.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB',
    dpi=100
)

from IPython.display import Image
Image('../results/model_architecture.png')

## 6. Treinamento

In [ ]:
# Cria callbacks
callbacks = criar_callbacks(
    model_dir='../models/notebook_model',
    patience=5,
    monitor='val_loss'
)

In [ ]:
# Treina o modelo
history = treinar_modelo(
    modelo,
    x_train_final, y_train_final,
    x_val, y_val,
    epochs=15,  # Reduzido para demonstração
    batch_size=32,
    callbacks=callbacks
)

In [ ]:
# Visualiza o histórico de treinamento
plotar_historico(history)

## 7. Avaliação

In [ ]:
# Avalia no conjunto de teste
test_loss, test_accuracy = avaliar_modelo(modelo, x_test_prep, y_test)

In [ ]:
# Faz predições
y_pred_probs, y_pred = fazer_predicoes(modelo, x_test_prep)

In [ ]:
# Matriz de confusão
matriz_confusao(y_test, y_pred, class_names)

In [ ]:
# Visualiza algumas predições
visualizar_predicoes(
    modelo,
    x_test_prep[:100],  # Primeiras 100 amostras
    y_test[:100],
    y_pred[:100],
    num_amostras=16,
    class_names=class_names
)

## 8. Salvamento e Carregamento do Modelo

In [ ]:
# Salva o modelo
from model import salvar_modelo, carregar_modelo

modelo_path = '../models/notebook_model/modelo_treinado.h5'
salvar_modelo(modelo, modelo_path)

In [ ]:
# Carrega o modelo salvo
modelo_carregado = carregar_modelo(modelo_path)

# Testa se funciona
test_loss_2, test_acc_2 = avaliar_modelo(modelo_carregado, x_test_prep[:100], y_test[:100])
print(f"\nModelo carregado funcionando corretamente!")

## Exemplo: Fazendo Predição em Uma Única Imagem

In [ ]:
# Pega uma imagem aleatória
idx = np.random.randint(0, len(x_test))
imagem = x_test_prep[idx:idx+1]  # Mantém dimensão do batch
label_real = y_test[idx]

# Faz a predição
predicao = modelo.predict(imagem)
classe_predita = np.argmax(predicao[0])
confianca = predicao[0][classe_predita]

# Visualiza
plt.figure(figsize=(6, 6))
plt.imshow(imagem[0].squeeze(), cmap='gray')
plt.title(f'Real: {class_names[label_real]}\nPredição: {class_names[classe_predita]} ({confianca:.2%})')
plt.axis('off')
plt.show()

# Mostra probabilidades para todas as classes
print("\nProbabilidades por classe:")
for i, prob in enumerate(predicao[0]):
    print(f"  {class_names[i]}: {prob:.4f}")

## Experimentos Adicionais

### 1. Teste com Diferentes Arquiteturas

In [ ]:
# Experimento: Modelo mais profundo
from tensorflow.keras import layers, models

def criar_modelo_profundo(input_shape, num_classes):
    """Modelo CNN mais profundo para experimentação"""
    model = models.Sequential([
        # Bloco 1
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloco 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Camadas densas
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Testa o modelo profundo
modelo_profundo = criar_modelo_profundo(input_shape, num_classes)
modelo_profundo.summary()

### 2. Data Augmentation

In [ ]:
# Cria pipeline de aumento de dados
from data_loader import criar_data_augmentation

data_aug = criar_data_augmentation()

# Visualiza efeitos do aumento de dados
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
axes = axes.ravel()

# Pega uma imagem
img = x_train_prep[0:1]

for i in range(8):
    if i == 0:
        axes[i].imshow(img[0].squeeze(), cmap='gray')
        axes[i].set_title('Original')
    else:
        augmented = data_aug(img)
        axes[i].imshow(augmented[0].numpy().squeeze(), cmap='gray')
        axes[i].set_title(f'Aumentada {i}')
    axes[i].axis('off')

plt.suptitle('Efeitos do Data Augmentation', fontsize=16)
plt.tight_layout()
plt.show()

## Como Adaptar para Outros Problemas

### 1. Para usar seus próprios dados:

```python
# Carregue suas imagens
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import glob

# Exemplo: carregar imagens de uma pasta
def carregar_imagens_pasta(pasta, tamanho=(28, 28)):
    imagens = []
    labels = []
    
    for classe, nome_classe in enumerate(os.listdir(pasta)):
        caminho_classe = os.path.join(pasta, nome_classe)
        for img_path in glob.glob(f"{caminho_classe}/*.jpg"):
            img = load_img(img_path, target_size=tamanho, color_mode='grayscale')
            img_array = img_to_array(img)
            imagens.append(img_array)
            labels.append(classe)
    
    return np.array(imagens), np.array(labels)
```

### 2. Para problemas de regressão:

```python
# Mude a última camada e a função de perda
model.add(layers.Dense(1))  # Sem ativação para regressão
model.compile(loss='mse', metrics=['mae'])
```

### 3. Para detecção de objetos ou segmentação:

- Use arquiteturas específicas como YOLO, R-CNN (detecção)
- Use U-Net, SegNet (segmentação)
- Adapte o carregamento de dados para incluir bounding boxes ou máscaras

## Recursos Adicionais

1. **TensorFlow Datasets**: Biblioteca com muitos datasets prontos
   ```python
   import tensorflow_datasets as tfds
   ds_train, ds_test = tfds.load('mnist', split=['train', 'test'])
   ```

2. **Transfer Learning**: Use modelos pré-treinados
   ```python
   base_model = tf.keras.applications.VGG16(
       input_shape=(224, 224, 3),
       include_top=False,
       weights='imagenet'
   )
   ```

3. **Callbacks Customizados**: Crie seus próprios callbacks
   ```python
   class MeuCallback(tf.keras.callbacks.Callback):
       def on_epoch_end(self, epoch, logs=None):
           print(f"Época {epoch} concluída!")
   ```